In [ ]:
import io
import tkinter as tk
from tkinter import ttk, filedialog
from PIL import Image, ImageTk
import numpy as np
import subprocess

# Load the TensorFlow Lite model (.tflite)
tflite_model_path = "/home/abhiram/CNN_TRAINED_MODEL.tflite"
interpreter = Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

# Load your mapping of class indices to class labels
class_labels = ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy',
                'Blueberry___healthy', 'Cherry_(including_sour)___healthy', 'Cherry_(including_sour)___Powdery_mildew',
                'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
                'Corn_(maize)___Common_rust_', 'Corn_(maize)___healthy', 'Corn_(maize)___Northern_Leaf_Blight',
                'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___healthy',
                'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
                'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy',
                'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight',
                'Potato___healthy', 'Potato___Late_blight',
                'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___healthy',
                'Strawberry___Leaf_scorch', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___healthy',
                'Tomato___Late_blight',
                'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite',
                'Tomato___Target_Spot', 'Tomato___Tomato_mosaic_virus', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus']

# Disease details
disease_details = {
    'Apple___Apple_scab': "To control Apple Scab, apply fungicides labeled for apple scab control. Additionally, prune and remove infected leaves.",
    'Apple___Black_rot': "For Black Rot in Apple trees, apply fungicides and practice good sanitation by promptly removing and destroying infected plant material.",
    'Apple___Cedar_apple_rust': "To manage Cedar Apple Rust, use fungicide treatments and consider removing cedar trees in the vicinity as they act as alternate hosts.",
    'Apple___healthy': "No specific treatment required for healthy apple trees.",
    'Blueberry___healthy': "No specific treatment required for healthy blueberry plants.",
    'Cherry_(including_sour)___healthy': "No specific treatment required for healthy cherry plants.",
    'Cherry_(including_sour)___Powdery_mildew': "To control Powdery Mildew in cherries, apply fungicides and maintain good air circulation.",
    'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot': "To manage Cercospora Leaf Spot and Gray Leaf Spot in corn, rotate crops and use fungicides.",
    'Corn_(maize)___Common_rust_': "For Common Rust in corn, use resistant varieties and apply fungicides if necessary.",
    'Corn_(maize)___healthy': "No specific treatment required for healthy corn plants.",
    'Corn_(maize)___Northern_Leaf_Blight': "To control Northern Leaf Blight in corn, practice crop rotation and use resistant varieties.",
    'Grape___Black_rot': "For Black Rot in grapes, apply fungicides and practice good sanitation by removing infected plant parts.",
    'Grape___Esca_(Black_Measles)': "To manage Esca (Black Measles) in grapes, prune infected vines and apply appropriate fungicides.",
    'Grape___healthy': "No specific treatment required for healthy grapevines.",
    'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)': "To control Leaf Blight in grapes, apply fungicides labeled for grape diseases.",
    'Orange___Haunglongbing_(Citrus_greening)': "There is no cure for Huanglongbing (Citrus Greening). Remove infected trees to prevent the spread.",
    'Peach___Bacterial_spot': "For Bacterial Spot in peaches, apply copper-based sprays and prune infected branches.",
    'Peach___healthy': "No specific treatment required for healthy peach trees.",
    'Peach___healthy': "No specific treatment required for healthy peach trees.",
    'Pepper,_bell___Bacterial_spot': "For Bacterial Spot in bell peppers, apply copper-based sprays and avoid overhead irrigation.",
    'Pepper,_bell___healthy': "No specific treatment required for healthy bell pepper plants.",
    'Potato___Early_blight': "To control Early Blight in potatoes, apply fungicides, practice crop rotation, and use disease-free seed potatoes.",
    'Potato___healthy': "No specific treatment required for healthy potato plants.",
    'Potato___Late_blight': "For Late Blight in potatoes, apply fungicides and avoid overhead irrigation.",
    'Raspberry___healthy': "No specific treatment required for healthy raspberry plants.",
    'Soybean___healthy': "No specific treatment required for healthy soybean plants.",
    'Squash___Powdery_mildew': "To control Powdery Mildew in squash, apply fungicides and maintain good air circulation.",
    'Strawberry___healthy': "No specific treatment required for healthy strawberry plants.",
    'Strawberry___Leaf_scorch': "To manage Leaf Scorch in strawberries, maintain good air circulation and remove infected leaves.",
    'Tomato___Bacterial_spot': "For Bacterial Spot in tomatoes, apply copper-based sprays and avoid overhead irrigation.",
    'Tomato___Early_blight': "To control Early Blight in tomatoes, apply fungicides, practice crop rotation, and use disease-free seedlings.",
    'Tomato___healthy': "No specific treatment required for healthy tomato plants.",
    'Tomato___Late_blight': "For Late Blight in tomatoes, apply fungicides and avoid overhead irrigation.",
    'Tomato___Leaf_Mold': "To manage Leaf Mold in tomatoes, apply fungicides and maintain good air circulation.",
    'Tomato___Septoria_leaf_spot': "For Septoria Leaf Spot in tomatoes, apply fungicides labeled for tomato diseases.",
    'Tomato___Spider_mites Two-spotted_spider_mite': "To control Spider Mites in tomatoes, use miticides and maintain a clean growing environment.",
    'Tomato___Target_Spot': "For Target Spot in tomatoes, apply fungicides labeled for tomato diseases.",
    'Tomato___Tomato_mosaic_virus': "There is no cure for Tomato Mosaic Virus. Remove infected plants to prevent the spread.",
    'Tomato___Tomato_Yellow_Leaf_Curl_Virus': "To manage Tomato Yellow Leaf Curl Virus, use disease-resistant varieties and control whiteflies.",
}

# List of classes that are considered as plants
plant_classes = ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy',
                'Blueberry___healthy', 'Cherry_(including_sour)___healthy', 'Cherry_(including_sour)___Powdery_mildew',
                'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
                'Corn_(maize)___Common_rust_', 'Corn_(maize)___healthy', 'Corn_(maize)___Northern_Leaf_Blight',
                'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___healthy',
                'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
                'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy',
                'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight',
                'Potato___healthy', 'Potato___Late_blight',
                'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___healthy',
                'Strawberry___Leaf_scorch', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___healthy',
                'Tomato___Late_blight',
                'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite',
                'Tomato___Target_Spot', 'Tomato___Tomato_mosaic_virus', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus']

def load_and_preprocess_image(img_path, target_size=(256, 256)):
    # Check if the image path is a URL
    if img_path.startswith("http://") or img_path.startswith("https://"):
        # Download the image from the URL
        response = requests.get(img_path)
        img = Image.open(io.BytesIO(response.content))
    else:
        # Load the local image
        img = Image.open(img_path)

    # Resize the image to match the expected input shape
    img = img.resize(target_size)
    img_array = np.array(img, dtype=np.float32) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    return img_array

def capture_image():
    # Capture image using raspistill command
    image_path = "captured_image.jpg"
    subprocess.run(["raspistill", "-o", image_path, "-w", "480", "-h", "320"])  # Adjust width and height

    # Display the captured image
    img = Image.open(image_path)
    img = ImageTk.PhotoImage(img)
    panel.config(image=img)
    panel.image = img

    # Set the image path in the entry widget
    img_path_entry.delete(0, tk.END)
    img_path_entry.insert(0, image_path)

    # Show the Perform Inference button
    perform_button.grid(row=4, column=0, pady=10, padx=5, sticky="ew")

def select_image():
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.jpeg;*.png")])
    img_path_entry.delete(0, tk.END)
    img_path_entry.insert(0, file_path)
    show_image(file_path)

    # Show the Perform Inference button
    perform_button.grid(row=4, column=0, pady=10, padx=5, sticky="ew")

def show_image(file_path):
    img = Image.open(file_path)
    img.thumbnail((480, 320))  # Adjusted size for the panel
    img = ImageTk.PhotoImage(img)
    panel.config(image=img)
    panel.image = img

def perform_inference(img_path_entry, result_label, status_label, warning_label, details_label):
    img_path = img_path_entry.get()

    if not img_path:
        status_label.config(text="Please select an image.")
        return

    img_array = load_and_preprocess_image(img_path, target_size=(256, 256))

    # Perform inference using the TensorFlow Lite model
    interpreter.set_tensor(interpreter.get_input_details()[0]['index'], img_array)
    interpreter.invoke()
    tflite_predictions = interpreter.get_tensor(interpreter.get_output_details()[0]['index'])

    predicted_class_index = np.argmax(tflite_predictions)
    predicted_class_label = class_labels[predicted_class_index]
    result_label.config(text="Predicted class: " + predicted_class_label)

    # Show disease details if available
    if predicted_class_label in disease_details:
        details = disease_details[predicted_class_label]
        details_label.config(text=details)
    else:
        details_label.config(text="Details not available for this disease.")

    # Check if the predicted class is a plant
    if predicted_class_label not in plant_classes:
        warning_label.config(text="Warning: Input may not be a plant.")
    else:
        warning_label.config(text="")

    status_label.config(text="Inference completed successfully.")

# Function to reset the GUI for another prediction
def reset_gui(img_path_entry, panel, result_label, details_label, status_label, warning_label):
    img_path_entry.delete(0, tk.END)
    panel.config(image="")
    result_label.config(text="")
    details_label.config(text="")
    status_label.config(text="")
    warning_label.config(text="")


def exit_application():
    root.destroy()

# Create GUI
root = tk.Tk()
root.title("Plant Disease Detection")
root.attributes("-fullscreen", True)  # Set to full screen

# Configure row and column weights
root.grid_rowconfigure(1, weight=1)
root.grid_columnconfigure(1, weight=1)

# GUI components
img_path_entry = ttk.Entry(root, width=20, font=("Helvetica", 10))
capture_button = ttk.Button(root, text="Capture Image", command=capture_image, width=15)
select_button = ttk.Button(root, text="Select Image", command=select_image, width=15)
result_label = ttk.Label(root, text="Predicted class: ", font=("Helvetica", 10))
details_label = ttk.Label(root, text="", font=("Helvetica", 10))
status_label = ttk.Label(root, text="")
warning_label = ttk.Label(root, text="", font=("Helvetica", 10))
perform_button = ttk.Button(root, text="Perform Inference",
                            command=lambda: perform_inference(img_path_entry, result_label, status_label, warning_label, details_label),
                            width=15)
reset_button = ttk.Button(root, text="Perform Another Prediction", command=lambda: reset_gui(img_path_entry, panel, result_label, details_label, status_label, warning_label), width=15)
panel = ttk.Label(root)
panel.grid(row=1, column=1, rowspan=6, padx=10, pady=10, sticky="nsew")
exit_button = ttk.Button(root, text="Exit", command=exit_application, width=15)
exit_button.grid(row=8, column=0, pady=10, sticky="ew")

# Grid layout
img_path_entry.grid(row=0, column=1, pady=10, padx=5, sticky="ew")
capture_button.grid(row=2, column=0, pady=5, padx=5, sticky="ew")
select_button.grid(row=3, column=0, pady=5, padx=5, sticky="ew")
result_label.grid(row=5, column=0, pady=5, padx=5, sticky="w")
details_label.grid(row=6, column=0, pady=5, padx=5, sticky="w")
status_label.grid(row=7, column=0, pady=5, padx=5, sticky="w")
warning_label.grid(row=8, column=0, pady=5, padx=5, sticky="w")
reset_button.grid(row=9, column=0, pady=10, padx=5, sticky="ew")

root.mainloop()
